In [1]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

# 2.4. Обучение и проверка моделей
Обучите Vowpal Wabbit на выборке stackoverflow_train.vw 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3). Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.
Проверяйте долю правильных ответов на выборке stackoverflow_valid.vw. Выберите лучшую модель и проверьте качество на выборке stackoverflow_test.vw.

 -oaa 10 – указываем, что классификация на 10 классов

-d – путь к данным
-f – путь к модели, которая будет построена
-b 28 – используем 28 бит для хэширования, то есть признаковое пространство ограничено 228 признаками, что в данном случае больше, чем число уникальных слов в выборке (но потом появятся би- и триграммы, и ограничение размерности признакового пространства начнет работать)
также указываем random seed
--quiet

In [2]:
from tqdm import tqdm

print('start')
pbar = tqdm()

best_score={}
def validation(text):#валидация
    pbar.update(1)
    !vw -t -i /ml/vw_model1_10mln.vw -d /ml/stackoverflow_valid.vw -p /ml/vw_valid_10mln_pred1.csv --random_seed 17 --quiet
    #оценка
    vw_pred = np.loadtxt('/ml/vw_valid_10mln_pred1.csv')
    test_labels = np.loadtxt('/ml/stackoverflow_valid_labels.txt')
    best_score[text]=accuracy_score(test_labels, vw_pred)
    print('accuracy_score ',text,': ',best_score[text])


#обучение
!vw --passes 1 --ngram 1 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 1 --ngram 1')

!vw --passes 3 --ngram 1 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 3 --ngram 1')

!vw --passes 5 --ngram 1 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 5 --ngram 1')

#best
!vw --passes 1 --ngram 2 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 1 --ngram 2')

!vw --passes 3 --ngram 2 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 3 --ngram 2')

!vw --passes 5 --ngram 2 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 5 --ngram 2')


!vw --passes 1 --ngram 3 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 1 --ngram 3')

!vw --passes 3 --ngram 3 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 3 --ngram 3')

!vw --passes 5 --ngram 3 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 5 --ngram 3')


pbar.close()
sv=sorted(best_score, key=best_score.get, reverse=True)
print('best - ',sv[0],':',round(best_score[sv[0]]*100,2))

0it [00:00, ?it/s]

start


1it [00:40, 40.91s/it]

accuracy_score  --passes 1 --ngram 1 :  0.915154153948


2it [03:30, 79.58s/it]

...

accuracy_score  --passes 1 --ngram 3 :  0.928603065718


8it [37:58, 382.10s/it]

accuracy_score  --passes 3 --ngram 3 :  0.926387781969


9it [49:19, 472.00s/it]

accuracy_score  --passes 5 --ngram 3 :  0.926167005464
--passes 1 --ngram 2 : 0.931083554679


In [3]:
print('Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке stackoverflow_valid.vw?')
print('Биграммы и 1 проход по выборке')

Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке stackoverflow_valid.vw?
Биграммы и 1 проход по выборке


In [ ]:
print('fit test')
!vw --passes 1 --ngram 2 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet

In [12]:
def score_test():
    print('valid test')
    !vw -t -i /ml/vw_model1_10mln.vw -d /ml/stackoverflow_test.vw -p /ml/vw_valid_10mln_pred1.csv --random_seed 17 --quiet

    #оценка
    print('score test')
    vw_pred = np.loadtxt('/ml/vw_valid_10mln_pred1.csv')
    test_labels = np.loadtxt('/ml/stackoverflow_test_labels.txt')
    val=accuracy_score(test_labels, vw_pred)
    print('accuracy_score test: ',round(val*100),2)   

In [ ]:
score_test()

print('best - ',sv[0],':',round(best_score[sv[0]]*100,2))
print('accuracy_score test: ',round(val*100,2))
round(val*100,2)-round(best_score[sv[0]]*100,2)<0.5

print('Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках?')
print('Результаты почти одинаковы – отличаются меньше чем на 0.5%')

valid
